In [17]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet50
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image
from torch.cuda import is_available, device_count

In [19]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

BATCH_SIZE = 64
EPOCHS = 200
LABEL_TO_CLASS: dict = {
    'N': 0,
    'D': 1,
    'G': 2,
    'C': 3,
    'A': 4,
    'H': 5,
    'M': 6,
    'O': 7
} 
NUM_CLASSES: int = len(LABEL_TO_CLASS)
DEVICE = device("cuda" if is_available() else "cpu")

NameError: name 'device' is not defined

In [16]:
# Define a SyntheticDataset class
class SyntheticDataset(Dataset):
    def __init__(self, root_dir, labels, transform=None):
        self.data = pd.read_csv(labels)
        self.root_dir = root_dir
        self.transform = transform
        self.label_mapping = {'N': 0,'D': 1,'G': 2,'C': 3,'A': 4,'H': 5,'M': 6,'O': 7}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name)

        label = self.label_mapping[self.data.iloc[idx, 1]]

        if self.transform:
            image = self.transform(image)

        return image, label
    
# Define DataLoader for synthetic dataset
synthetic_dataset = SyntheticDataset(root_dir='../data/content/aug_balanced_imgs', labels='../data/balanced_odir_labels.csv', transform=transform)
synthetic_dataloader = DataLoader(synthetic_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Define image classification model
model = resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)
model = model.to(DEVICE)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training loop
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(synthetic_dataloader):
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE).squeeze().long()
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:  # Print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

/Users/sarahcasale/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/sarahcasale/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


NameError: name 'DEVICE' is not defined